In [2]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import MCXGate

In [35]:
def createRemoverCircuits(de,ext,delta,K,B)-> list: # will create remover circuits for 
    removeCircuits=[] 

    ctrl_bits=[de+ext-1-i-B for i in range(K-1,-1,-1)]+[i for i in range(2,2+K)]+[0]
    # print(ctrl_bits,de,ext,B)
    for i in range(delta,-1,-1):
        circuit= QuantumCircuit(de+ext) ## Creating new quantum circuit (remover{i})
        bini=bin(delta-i)[2:].rjust(K,'0')
        ## adding control on value of above K elements from value delta to 0 and then appending the remover circuits
        for j in range(0,i):
            binaryOfj=bini+bin(delta-j)[2:].rjust(K,'0')
            circuit.append(MCXGate(num_ctrl_qubits=K+K,ctrl_state=binaryOfj),ctrl_bits)
            circuit=removeAll(circuit,de,ext)

        removeCircuits.append(circuit)

    return removeCircuits

## remover circuit
def removeAll(qc:QuantumCircuit,de,ext)-> QuantumCircuit:
    for i in range(de):
        qc.cswap(0,i+ext,1)
        qc.reset(1)

    qc.reset(0)
    return qc


## Oracle

def checkerOracle2(qc:QuantumCircuit,totalqb,B,searchNode): # 
    qc.reset(2)
    print(totalqb,[totalqb-i for i in range(B,0,-1)]+[2],qc.num_qubits)
    qc.append(MCXGate(num_ctrl_qubits=B,ctrl_state=searchNode),[totalqb-i for i in range(B,0,-1)]+[2])
    return qc

def Oracle2(qc:QuantumCircuit,de,ext,K,B,delta,searchNode): # 
    qc=checkerOracle2(qc,de+ext,B,searchNode)
    qc.measure(2,2)
    with qc.if_test((2,1)):
        qc.reset([i for i in range(2,K+2)])
    qc.reset(2)
    return qc



In [49]:
## Not complete
K=2
B=3
max_Depth=5
delta=3
ext=4
de=2*(K+B)+ext
removerCircuits=createRemoverCircuits(de,ext,delta,K,B)
NoOfEdges={"A":3,"B":1,"C":2,"D":0}

def checkerOracle2(qc:QuantumCircuit,totalqb,B,searchNode):
    qc.reset(2)
    print(totalqb,[totalqb-i for i in range(B,0,-1)]+[2],qc.num_qubits)
    qc.append(MCXGate(num_ctrl_qubits=B,ctrl_state=searchNode),[totalqb-i for i in range(B,0,-1)]+[2])
    return qc

def Oracle2(qc:QuantumCircuit,de,ext,K,B,delta,searchNode):
    qc=checkerOracle2(qc,de+ext,B,searchNode)
    qc.measure(2,2)
    with qc.if_test((2,1)):
        qc.reset([i for i in range(2,K+2)])
    qc.reset(2)
    return qc





[13, 14, 2, 3, 0] 14 4 3


In [ ]:
circuit= QuantumCircuit(de+ext,3) 
from qiskit import *
# backend = Aer.get_backend('_simulator')
# circuit=transpile(circuit,backend)
# result= backend.run(circuit).result()

circuit=Oracle2(circuit,de,ext,K,B,delta,"001")
circuit.draw()

In [ ]:
removerCircuits[1].draw()

# Time Complexity

### Our Algorithm-->


#Total : 

### Classical -->
K*K\*K....m times
=~ *K*<sup>m</sup>

In [56]:
'''
This code is currently incomplete (oracle, checker is left to be done).
Rem function (remover) is not in this file, it will be added later (probably by priyanshu or pranav).
Proceed with caution.
'''

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import XGate,MCXGate
from qiskit.circuit import AncillaQubit,Qubit
import numpy as np
import remover


delta = 6 # delta = max no of edges
maxde = 10 # maximum depth
count = 0
K = int(np.ceil(np.log2(delta+1))) # Getting no of bits required to represent delta+1, K = ceil(log2(delta + 1))
ext = 2 # used by remover

qK = QuantumRegister(K) # branching qubits
rem = QuantumRegister(ext) # required for remover
orc = QuantumRegister(K) # required for oracle
B = QuantumRegister(3) # represents the actual value
measuringReg= ClassicalRegister(3) # represents classical register
de = (2*K+B.size)# de = (K+B)*maxde (max no of bits)

ctrlBits=[i for i in range(de+ext)]


remCirs = []

def init():
    qc = QuantumCircuit(rem,orc,qK,B,measuringReg)
    qc.x(rem.size+orc.size)
    for i in qK: # O(K)
        qc.h(i)
    for i in B: # O(B)
        qc.h(i)
    
    return qc

def calc(qc:QuantumCircuit,remCircuits): # 1 iteration - O(maxde*(de+K+?))
    oracle(qc) # O(?)
    
    for i in range(1, de - 1 - delta): # O(de - delta)
        qc.append(remCircuits[i],list(range(qc.num_qubits)))
    q = QuantumCircuit(qc.num_qubits+qK.size+B.size).compose(qc,list(range(qc.num_qubits)))
    # Reassign remover circuit
    remCircuits = createRemoverCircuits(de,ext,delta,K,B.size) # list of remove circuits
    
    # count+=1
    for i in range(qc.num_qubits,qc.num_qubits+qK.size): # O(K)
        q.h(i)
    
    for cir in remCircuits:
        qc.append(cir,ctrlBits)
        
    return qc
    # calc(q)

def oracle(qc): # check the qc if in target state (yes, then remove all), get current node edges
    # if(qc in target_conditions): # target conditions left to figure out
    #     v = rem.size + orc.size + ((qc.num_qubits - rem.size - orc.size)//(qK.size+B.size) -1)*(qK.size+B.size)-1
    #     n = node(v) # given the qubits of B, return the corresponding vertex
    #     X = # no of edges of current node left to figure out, get edges from n
    #     qc.append(removeCircuit[K-X],qc.num_qubits) #


    qc=Oracle2(qc,de,ext,K,B.size,delta,searchNode="001")

def checker(i):
    ck = QuantumCircuit(K+1)
    for i in range(K):
        ck.x(i)
    ck.append(XGate().control(list(range(K))),QuantumRegister(ck.num_qubits))
    for i in range(K):
        ck.x(i)
    return ck.to_gate()


def node(n): # get the value of the last B register
    anc = ClassicalRegister(1)
    w = QuantumRegister(n)
    nc = QuantumCircuit(w + 1, anc)
    nc.reset(n)
    nc.append(checker(0), list(range(K)) + [n])
    nc.measure(n, anc[0])
    with nc.if_test((anc[0], 1)) as else_:
        for i in range(K):
            nc.reset(n)
            nc.append(checker(i), list(range(K, 2 * K)) + [n])
            nc.measure(n, anc[0])
            with nc.if_test((anc[0], 1)):
                pass

    with else_:
        pass


qc = init()
removeCircuits = createRemoverCircuits(de,ext,delta,K,B.size) # list of remove circuits
qc=calc(qc,removeCircuits)
qc.draw()
# calc(qc)

[5, 6, 7, 2, 3, 4, 0] 9 2 3
11 [8, 9, 10, 2] 11
[5, 6, 7, 2, 3, 4, 0] 9 2 3


┌───────────────┐»
q125_0: ───────────────────────────────────────────────────┤0              ├»
                                                           │               │»
q125_1: ───────────────────────────────────────────────────┤1              ├»
                  ┌───┐┌─┐   ┌──────         ───────┐      │               │»
q126_0: ─|0>──────┤ X ├┤M├───┤       ───|0>─        ├──|0>─┤2              ├»
                  └─┬─┘└╥┘   │                      │      │               │»
q126_1: ────────────┼───╫────┤ If-0  ───|0>─  End-0 ├──────┤3              ├»
                    │   ║    │                      │      │               │»
q126_2: ────────────┼───╫────┤       ───|0>─        ├──────┤4              ├»
        ┌───┐┌───┐  │   ║    └──╥───         ───────┘      │               │»
q124_0: ┤ X ├┤ H ├──┼───╫───────╫──────────────────────────┤5  circuit-472 ├»
        ├───┤└───┘  │   ║       ║                          │               │»
q124_1: ┤ H ├───────┼───╫───────╫──────────────────────────┤6              ├»
        ├───┤       │   ║       ║                          │               │»
q124_2: ┤ H ├───────┼───╫───────╫──────────────────────────┤7              ├»
        ├───┤       │   ║       ║                          │               │»
q127_0: ┤ H ├───────■───╫───────╫──────────────────────────┤8              ├»
        ├───┤       │   ║       ║                          │               │»
q127_1: ┤ H ├───────o───╫───────╫──────────────────────────┤9              ├»
        ├───┤       │   ║       ║                          │               │»
q127_2: ┤ H ├───────o───╫───────╫──────────────────────────┤10             ├»
        └───┘           ║ ┌─────╨─────┐                    └───────────────┘»
 c20: 3/════════════════╩═╡ c20_2=0x1 ╞═════════════════════════════════════»
                        2 └───────────┘                                     »
«        ┌───────────────┐┌───────────────┐┌───────────────┐┌───────────────┐»
«q125_0: ┤0              ├┤0              ├┤0              ├┤0              ├»
«        │               ││               ││               ││               │»
«q125_1: ┤1              ├┤1              ├┤1              ├┤1              ├»
«        │               ││               ││               ││               │»
«q126_0: ┤2              ├┤2              ├┤2              ├┤2              ├»
«        │               ││               ││               ││               │»
«q126_1: ┤3              ├┤3              ├┤3              ├┤3              ├»
«        │               ││               ││               ││               │»
«q126_2: ┤4              ├┤4              ├┤4              ├┤4              ├»
«        │               ││               ││               ││               │»
«q124_0: ┤5  circuit-482 ├┤5  circuit-483 ├┤5  circuit-484 ├┤5  circuit-485 ├»
«        │               ││               ││               ││               │»
«q124_1: ┤6              ├┤6              ├┤6              ├┤6              ├»
«        │               ││               ││               ││               │»
«q124_2: ┤7              ├┤7              ├┤7              ├┤7              ├»
«        │               ││               ││               ││               │»
«q127_0: ┤8              ├┤8              ├┤8              ├┤8              ├»
«        │               ││               ││               ││               │»
«q127_1: ┤9              ├┤9              ├┤9              ├┤9              ├»
«        │               ││               ││               ││               │»
«q127_2: ┤10             ├┤10             ├┤10             ├┤10             ├»
«        └───────────────┘└───────────────┘└───────────────┘└───────────────┘»
« c20: 3/════════════════════════════════════════════════════════════════════»
«                                                                            »
«        ┌───────────────┐┌───────────────┐┌───────────────┐
«q125_0: ┤0              ├┤0              ├┤0              ├
«        │  